In [ ]:
from __future__ import print_function
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Convolution2D, GlobalAveragePooling2D
from tensorflow.keras import layers

from keras.utils import np_utils
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
# from tensorflow.keras.layers.core import Lambda
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas
import cv2
import numpy as np

K.set_image_data_format('channels_last')

def set_tensorflow_config(per_process_gpu_memory_fraction=0.8):
    config = tf.compat.v1.ConfigProto()
    # config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = per_process_gpu_memory_fraction
    config.gpu_options.allow_growth=True
    # sess = tf.Session(config=config)
    sess = tf.compat.v1.Session(config=config)
    
    print("== TensorFlow Config options set ==")
    print("\nThis process will now utilize {} GPU Memeory Fraction".format(per_process_gpu_memory_fraction))

set_tensorflow_config()


batch_size = 32
nb_classes = 10
nb_epoch = 350


rows, cols = 32, 32

channels = 3

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

print (X_train.shape[1:])

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = keras.models.Sequential([
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', input_shape=(32,32,3)),
    keras.layers.Activation('relu'),
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same'),
    keras.layers.Activation('relu'),
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(2,2), padding='same'),
    keras.layers.Dropout(0.2),

    keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same'),
    keras.layers.Activation('relu'),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same'),
    keras.layers.Activation('relu'),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(2,2), padding='same'),
    keras.layers.Dropout(0.2),
    
    keras.layers.Conv2D(filters=192, kernel_size=(3,3), padding='same'),
    keras.layers.Activation('relu'),
    keras.layers.Conv2D(filters=192, kernel_size=(3,3), padding='same'),
    keras.layers.Activation('relu'),
    keras.layers.Conv2D(filters=192, kernel_size=(3,3), strides=(2,2), padding='same'),
    keras.layers.Dropout(0.2),
    
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same'),
    keras.layers.Activation('relu'),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), padding='valid'),
    keras.layers.Activation('relu'),
    keras.layers.Conv2D(filters=10, kernel_size=(1,1), padding='valid'),   
])


model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Softmax())

sgd = keras.optimizers.SGD(learning_rate=0.005, momentum=0.91, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

print (model.summary())

model_json = model.to_json()
with open("allcnn_model.json", "w") as json_file:
    json_file.write(model_json)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True) 

datagen.fit(X_train)
filepath="allcnn_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='max')

callbacks_list = [checkpoint]
    # Fit the model on the batches generated by datagen.flow().
history_callback = model.fit(datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        steps_per_epoch=1563,
                        epochs=50,
                        validation_data=(X_test, Y_test),
                        callbacks=callbacks_list, verbose=1)


== TensorFlow Config options set ==

This process will now utilize 0.8 GPU Memeory Fraction
X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
(32, 32, 3)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
activation (Activation)      (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
dropout (Dr

In [ ]:
# im = cv2.resize(cv2.imread('image.jpg'), (224, 224)).astype(np.float32)
# out = model.predict(im)
# print np.argmax(out)

# pandas.DataFrame(history_callback.history).to_csv("history.csv")